In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("PySpark Intro") \
    .getOrCreate()

In [ ]:
tweets = spark.read.options(samplingRatio=0.01).json('gs://bgse-datawarehousing-random-tweets')
tweets.printSchema()

In [ ]:
import itertools as it
import numpy as np
from scipy import sparse
import pandas as pd

intermediate = tweets.rdd \
    .filter(lambda t: (t.entities is not None) and (len(t.entities.hashtags)>=2)) \
    .map(lambda t: [k.text.lower() for k in t.entities.hashtags]) \
    .flatMap(lambda t: it.permutations(t,2)) \
    .map(lambda t: (t,1)) \
    .reduceByKey(lambda a,b:a+b).cache()
index_list = intermediate \
    .map(lambda t: t[0][0]) \
    .collect()
dict_index = dict.fromkeys(index_list)
val = 0
for i in dict_index:
    dict_index[i] = val
    val += 1

In [ ]:
matrix = intermediate \
    .map(lambda t: [t[0][0], t[0][1], t[1]]) \
    .map(lambda t: (dict_index[t[0]],dict_index[t[1]],t[2])) \
    .map(lambda t: sparse.coo_matrix(([t[2]], ([t[0]], [t[1]])), shape = (len(dict_index),len(dict_index)))) \
    .reduce(lambda a,b: a+b)